In [1]:
import ChatTTS

chat = ChatTTS.Chat()
chat.load(compile=False, source='custom', custom_path='/root/Project_FunGPT/Developing/TTS/weights/ChatTTS')  # 用这个最新的配置

/root/.conda/envs/FunGPT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
use default LlamaModel for importing TELlamaModel error: No module named 'transformer_engine'
/root/.conda/envs/FunGPT/lib/python3.10/site-packages/ChatTTS/core.py:327: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode un

True

In [2]:
texts = [
    "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with.",
] * 3 + [
    "我觉得像我们这些写程序的人，他，我觉得多多少少可能会对开源有一种情怀在吧我觉得开源是一个很好的形式。现在其实最先进的技术掌握在一些公司的手里的话，就他们并不会轻易的开放给所有的人用。"
] * 3

wavs = chat.infer(texts)

text:   0%|          | 1/384(max) [00:02,  2.94s/it]We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
text:  53%|█████▎    | 203/384(max) [01:06,  3.07it/s]
code:  54%|█████▍    | 1101/2048(max) [02:41,  6.84it/s]


In [4]:
from IPython.display import Audio

In [7]:
import numpy as np
np.array(wavs[0])

array([[-1.45359931e-03, -1.39254320e-03, -1.35344733e-03, ...,
        -1.21511126e-04, -1.39836004e-04, -5.81956629e-05]], dtype=float32)

In [8]:
wavs[0]

array([[-1.45359931e-03, -1.39254320e-03, -1.35344733e-03, ...,
        -1.21511126e-04, -1.39836004e-04, -5.81956629e-05]], dtype=float32)

In [13]:
import soundfile as sf
sf.write('a.wav', wavs[0], samplerate=24000)

LibsndfileError: Error opening 'a.wav': Format not recognised.

In [9]:
Audio(np.array(wavs[0]), rate=24_000, autoplay=True)

In [ ]:
Audio(wavs[3], rate=24_000, autoplay=True)

### Custom params

In [ ]:
params_infer_code = ChatTTS.Chat.InferCodeParams(
    prompt="[speed_5]",
    temperature=0.3,
)
params_refine_text = ChatTTS.Chat.RefineTextParams(
    prompt="[oral_2][laugh_0][break_6]",
)

wav = chat.infer(
    "四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片等，每样都让人垂涎三尺。",
    params_refine_text=params_refine_text,
    params_infer_code=params_infer_code,
)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Fix random speaker

In [ ]:
rand_spk = chat.sample_random_speaker()
print(rand_spk)  # save it for later timbre recovery

params_infer_code = ChatTTS.Chat.InferCodeParams(
    spk_emb=rand_spk,
)

wav = chat.infer(
    "四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。",
    params_infer_code=params_infer_code,
)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Zero shot (simulate speaker)

In [ ]:
from tools.audio import load_audio

spk_smp = chat.sample_audio_speaker(load_audio("sample.mp3", 24000))
print(spk_smp)  # save it in order to load the speaker without sample audio next time

params_infer_code = ChatTTS.Chat.InferCodeParams(
    spk_smp=spk_smp,
    txt_smp="与sample.mp3内容完全一致的文本转写。",
)

wav = chat.infer(
    "四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。",
    params_infer_code=params_infer_code,
)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Two stage control

In [ ]:
text = "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with."
refined_text = chat.infer(text, refine_text_only=True)
refined_text

In [ ]:
wav = chat.infer(refined_text, skip_refine_text=True)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

## LLM Call

In [ ]:
from tools.llm import ChatOpenAI

API_KEY = ""
client = ChatOpenAI(
    api_key=API_KEY, base_url="https://api.deepseek.com", model="deepseek-chat"
)

In [ ]:
user_question = "四川有哪些好吃的美食呢?"

In [ ]:
text = client.call(user_question, prompt_version="deepseek")
text

In [ ]:
text = client.call(text, prompt_version="deepseek_TN")
text

In [ ]:
wav = chat.infer(text)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)